<a href="https://colab.research.google.com/github/natnj/NJ-projects-pub/blob/main/MLCB_Homework_3_NJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLCB Homework 3

**Due: Monday Nov 6th at 11:59 PM EDT**

This homework has two parts. In the first part, you will work with gene expression data and identify eQTLs.

The second part will focus on protein modeling, the goal being to expose you to different tools and data that can help you in your final project. In particular you will be using:

- The RCSB PDB database (protein structures)
- The ESM protein language model from Meta
- AlphaFold2
- ESMFold
- ProteinMPNN (optional for undergads)

If you have any questions, please ask on piazza or at office hours!

Copy this notebook, and answer all questions directly in this notebook and complete the missing code where marked with **COMPLETE HERE**.

When you are done, submit the .ipynb file as well as PDF to Canvas.

## 0. Setup (run this)

In [ ]:
!pip install fair-esm
!pip install scikit-learn
!pip install biopython
!pip install matplotlib
!pip install torch
!pip install pandas
!pip install tqdm
!wget -O train_esm6_data.pt https://www.dropbox.com/scl/fi/cf103vkoew0s5zys7920d/train_esm6_data.pt?rlkey=v7o2upz9txm0m284g6mgfq0ww&dl=0
!wget -O test_esm6_data.pt https://www.dropbox.com/scl/fi/v18bvl57eog1ysjb07nww/test_esm6_data.pt?rlkey=idc0y0nzwfizxxf75kyxbfxdh&dl=0
!wget -O train_esm150_data.pt https://www.dropbox.com/scl/fi/i4vazz4twcw38wzaykdq0/train_esm150_data.pt?rlkey=ev7tepsceqc4k5vbwsw5bsw67&dl=0
!wget -O test_esm150_data.pt https://www.dropbox.com/scl/fi/yyjk31urec4gdwk8ajppt/test_esm150_data.pt?rlkey=v8o5mzp608w8tmxywqmn4r3ob&dl=0

import matplotlib.pyplot as plt
import esm
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
from Bio.PDB import PDBParser, PDBList
from google.colab import files
import os
from sklearn.decomposition import PCA

esm6_model, esm6_alphabet = esm.pretrained.esm2_t6_8M_UR50D()
esm6_model.eval()

esm150_model, esm150_alphabet = esm.pretrained.esm2_t30_150M_UR50D()
esm150_model.eval()

AA_NAME_MAP = {
  'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
  'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
  'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 'TER':'*',
  'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M','XAA':'X'
}

## 1. Finding eQTLs

In this problem, we will examine the sources of variation in gene expression that partition a population into subpopulations.

You will access and upload them to colab using the following code block.

In [ ]:
!wget -c https://www.dropbox.com/sh/9aozrd1fv0bg1xd/AACCM0Y4J9fFTSv9YSjrKO1ya?dl=0 -O eqtl_data.zip
!unzip -o eqtl_data.zip -d eqtl_data
os.unlink("eqtl_data.zip")
eqtl_data = {}
for fname in os.listdir("eqtl_data"):
    with open(os.path.join("eqtl_data", fname), "r") as R:
        eqtl_data[fname] = R.read().encode('utf-8')

You are free to use the following function to help you process the raw `ExpData.txt` and `SnpData.txt` files.

In [ ]:
# Processes *Data.txt files
"""
Example inp:
    Patient Trait1 Trait2 Trait3
    A 0.1 0.2 0.3
    B 0.4 0.5 0.6
    C 0.7 0.8 0.9

process_file(inp, "patient")
    [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]]

process_file(inp, "trait")
    {"Trait1": [0.1, 0.4, 0.7], "Trait2": [0.2, 0.5, 0.8], "Trait3": [0.3, 0.6, 0.9]}
"""
def process_file(inp, mode):
    rows = inp.decode("utf-8").split("\n")
    if mode == "patient":
        return [[float(val) for val in row.split()[1:]] for row in rows[1:] if row]
    elif mode == "trait":
        res = pd.DataFrame([row.split()[1:] for row in rows if row])
        res.columns = res.iloc[0]
        return {k: [float(val) for val in v.values()] for k, v in res[1:].to_dict().items()}

#process_file(eqtl_data["SnpData.txt"], "patient")
#process_file(eqtl_data["SnpData.txt"], "trait")
#process_file(eqtl_data["ExpData.txt"], "patient")
#process_file(eqtl_data["ExpData.txt"], "trait")

Use the following code block if you would like to process the raw `ExpData.txt` and `SnpData.txt` files in your own way.

In [ ]:
### COMPLETE HERE ###

A) The `ExpData.txt` file contains log-normalized RNA-seq expression data from our population of 1,000 samples, with 5,000 genes profiled for each sample. Do a principal components analysis on this dataset to find the clusters of samples that have similar patterns of gene expression. Plot the output of your analysis.
In your plots, be sure the axes are labeled with the components you are displaying in each plot. Also make sure that at least one of your plots colours the points corresponding to the samples with the sub-population that you think they should belong to. (Hint: You can re-use your $k$-means code from PSET 1 to find these sub-populations!)

Describe the patterns that you observe. What is the structure inherent in this population?

You may find the `matplotlib.pyplot` and `sklearn.decomposition.PCA` packages (already imported) to be useful.

In [ ]:
### COMPELTE HERE ###

**Answer:**

B) The `SnpData.txt` file contains genotyping data for the same 1,000 samples across 500 SNPs. Each SNP's genotype has been called with reference to the same reference genotype; "0" thus represents the reference allele, "2" represents the non-reference allele, and "1" represents a different allele on each strand.

You will find that some of the SNPs (more than 5, less than 100) are eQTLs, that is, they have an effect on the expression of one or more of the genes we collected expression data for. Using whatever model you see fit, search for these eQTLs using the genotyping data and the expression data. **You may not have the computational resources to test all combinations of SNPs and genes, so you should think about smart ways to choose subsets of each to find some eQTLs - you don't have to find all of them!**

For three of the eQTLs you found, present the evidence you have for why you think it is an eQTL, and not just associated with the expression of a gene by chance alone. *Be sure to include plots in your analysis to support your hypothesis, and to thoroughly explain the method you used to find eQTLs.* You can assume that the association between genotype and expression is linear for eQTLs. *Don't forget that you should be correcting for the fact that you are performing multiple significance tests.*

In [ ]:
### COMPLETE HERE ###

**Answer:**

eQTL 1:

eQTL 2:

eQTL 3:

C) In the above analysis, we were forced to consider all pairs of SNPs and genes to identify eQTLs. What sources of data that have not been provided as part of this problem would have been useful in constraining the amount of such pairs you had to test? For at least two sources:

i. Give a description of what the dataset would look like (i.e. what are the rows and columns of the data matrix? what kinds of values are stored in the matrix?).

ii. Explain how you would use it to filter out pairs of SNPs and genes that are unlikely to be associated with one another.

**Answer**:

Source 1:

i.

ii.

Source 2:

i.

ii.

## 2. Protein Language Models

In this problem we will explore protein language models. The goal is to give you some intuitions as to what these models learn during the pretraining process and how they can be used in downstream applications.

Protein language models are trained using the masked language modeling objective: given an input sequence, a subset of amino acids are masked and the model is tasked with predicting the masked tokens.

As we will see below, this allows the network to learns complex features of proteins.

### 2.A Amino Acid Embeddings

In this first part, we will explore how PLM's learn embeddings (i.e vectors of features) that represent amino acids.

#### 2.A.1 Question

Here we provide you with a vector for each amino acid. Run a dimensionality reduction startegy (ex: t-SNE) to visualize the amino acids in 2D. You can use whatever method you see fit but if you need help getting started, check out this link: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

Note: when using the TSNE implementation above, you may want to use a small value for the perplexity, 3 seemed to work well.



In [ ]:
# Create a dictionary mapping an amino acid to its vector
def get_aa_embeddings(model, alphabet):
  embedding_matrix = model.embed_tokens.weight.data.numpy()
  aa_tokens = alphabet.standard_toks[:-2]
  aa_to_index = alphabet.to_dict()
  aa_to_embeddings = {aa: embedding_matrix[aa_to_index[aa]] for aa in aa_tokens}
  return aa_to_embeddings


# COMPLETE HERE
# Reduce to 2D using t-SNE or other technique and inspect visually
# You can do it for both models and compare!
esm6_aa_embeddings = get_aa_embeddings(esm6_model, esm6_alphabet)
esm150_aa_embeddings = get_aa_embeddings(esm150_model, esm150_alphabet)


#### 2.A.2 Question

Comment on your observations. In particular, pick 2 clusters and identify what is similar about the amino acids in these clusters (think of their chemical properties). You may refer to Wikipedia and other sources for information regarding the chemical properties of amino acids. No need to run a clustering algorithm, just visually inspect. How do the projections compare between the two models?

**Answer here**:

### 2.B Sequence embeddings

In this second part, we will use sequence level embeddings and build a simple classifier to predict protein sub-cellular location, specifically if a protein is a membrane-protein or not.

#### 2.B.1 Question

Embeddings were precomputed for you. All you need to do is train a classifier. Here you are welcome to go as complex as you want to, but the simplest approach would be to train a logistic regression model using sklearn:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

When using LogisticRegression, set C=10 and max_iter=1000 for best results.

Make sure to train on the training set, and evaluate your performance on the test set.

In [ ]:
train_esm6 = torch.load("/content/train_esm6_data.pt")
test_esm6 = torch.load("/content/test_esm6_data.pt")

train_esm150 = torch.load("/content/train_esm150_data.pt")
test_esm150 = torch.load("/content/test_esm150_data.pt")

# COMPLETE HERE
# Train a classifier using whatever tool you prefer (simplest is sklearn's logisitc regression above!)
# When using LogisticRegression, set C=10 and max_iter=1000 for best results
# The data is provided as a list of tuples (vector, label)

#### 2.B.2 Question

Compute both accuracy and ROC-AUC (see sklearn's roc_auc_score). Which model performed best on the test set?

**ANSWER**:

## 3. Protein Structure Prediction

In this problem, you will get some experience working with structure prediction models.

### 3.A Contact Prediction

One way to get a rough idea of a protein's 3D structure is to look at its contact map. Pick your favorite protein from the PDB (https://www.rcsb.org/). If the protein has multiple chain, specificy which chain should be loaded. Make sure it isn't too long, it should be less than 400 amino acids. Also try to avoid proteins with missing residues (faded letters in the sequence when using the 3D view on the rcsb website). If you get an error loading the PDB, try a different one. If you struggle to find one that works, you may use for example "1A0K".

#### 3.A.1 Question

Which protein did you choose and why?

**Answer**:

#### 3.A.2 Question

Using the code below, load the protein sequence and its structure.

Then complete the code to compute the pairwise contacts between the residues.

In [ ]:
# COMPLETE HERE
# If you get an error, try a different PDB entry
PDB_ID = "" # The PDB ID of the protein you chose
PDB_CHAIN = "" # The chain to load (you can view chains in the 3D view on rcsb.org)


def compute_contacts(coords, threshold=9.0):
  """Compute the pairwise contacts.

  Here we define a contact as the C-alpha atom
  of 2 amino acids being within 9 Angstrom of each other.

  Parameters
  ----------
  coords:
    array of shape (num_residues, 3)
  threshold:
    distance threshold to consider a contact

  Returns
  -------
  contacts:
    binary array of shape (num_residues, num_residues)

  """
  # COMPLETE HERE
  pass


# DON'T MODIFY BELOW
def load_pdb(pdb_id, chain_id):
  pdbl = PDBList()
  pdbp = PDBParser()
  pdbl.retrieve_pdb_file(pdb_id.upper(), file_format="pdb", pdir="./")
  struct = pdbp.get_structure("struct", f"pdb{pdb_id.lower()}.ent")
  model = struct[0]

  sequence = []
  coords = []
  for chain in model:
    if chain.id != chain_id:
      continue
    for residue in chain:
      if residue.resname == "HOH":
        continue
      try:
        coords.append(residue['CA'].get_vector().get_array())
      except:
        raise ValueError("There are missing atoms in this structure, try another!")
      resname = AA_NAME_MAP[residue.resname]
      sequence.append(resname)

  return "".join(sequence), np.array(coords)


pdb_seq, coords = load_pdb(PDB_ID, PDB_CHAIN)
contacts = compute_contacts(coords)
plt.imshow(contacts)


#### 3.A.3 Question

Now we will compute the predicted contacts using our PLM. Compare the predicted contacts and the ground truth contacts. How well do they match up? Inspect visually by plotting them using plt.imshow, and then compare using the mean average precision metric from sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html.

In [ ]:
from sklearn.metrics import average_precision_score


def predict_contacts(model, alphabet, sequence):
  """Predict contacts with ESM."""
  batch_converter = alphabet.get_batch_converter()
  data = [("prot", sequence)]
  batch_labels, batch_strs, batch_tokens = batch_converter(data)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

  with torch.no_grad():
      num_layers = model.num_layers
      results = model(batch_tokens, repr_layers=[num_layers], return_contacts=True)
  token_representations = results["representations"][num_layers]

  tokens_len = batch_lens[0]
  attention_contacts = results["contacts"][0]
  return attention_contacts[: tokens_len, : tokens_len]


def compare_contacts(pred_contacts, true_contacts):
  """Compares two contact maps.

  Prints the average precision and plots both maps.

  Parameters
  ----------
  pred_contacts:
    array of shape (num_residues, num_residues)
  true_contacts:
    array of shape (num_residues, num_residues)

  """
  # COMPLETE HERE
  # Make sure to flatten before computing the average precision
  pass

# we use the pdb_seq and contacts variables from the previous question
pred_contacts = predict_contacts(esm6_model, esm6_alphabet, pdb_seq)
compare_contacts(pred_contacts, contacts)



#### 3.A.4 Question

Try the same experiment with the esm150_model. Which performs best?

In [ ]:
# COMPLETE HERE
# Try the esm150_model and the esm150_alphabet

**ANSWER**:


### 3.B Structure Prediction

In this problem, you will pick a protein sequence of interest and attempt to fold it using both AlphaFold2 and ESMFold. Pick from the PDB as before. You may want to filter to recently deposited structures since older ones will be in the training sets of AF2 and ESMFold. Make sure it's under 400 amino acids and doesn't contain missing residues.

For instance you may use this one:
PDB_ID = 8CH7, CHAIN_ID = "A" (https://www.rcsb.org/3d-view/8BY4)


#### 3.B.1 Question

First let's load the ground truth structure similarly to before. Print the sequence, you will need it for the next questions.

In [ ]:
# COMPLETE HERE
# Feel free to use a different PDB!
PDB_ID = "8CH7"
PDB_CHAIN = "A"
pdb_seq, true_coords = load_pdb(PDB_ID, PDB_CHAIN)
print(pdb_seq)


#### 3.B.2 Question

Run your protein through AlphaFold2 using this notebook:
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb

All you need to do is paste in your sequence from above and run all the cells. No need to modify any of the parameters. See the part that says:  
`Input protein sequence(s), then hit Runtime -> Run all`

This may take several minutes to complete. Once done, take a screenshot of the predicted structure and download results. The external notebook downloads zip file `jobname.result.zip` to your system. On extracting, among many other files, there are a sequence of PDB files out of which the one that is first generated can be uploaded. It will look something like this:  
`<job_name>_unrelaxed_rank_005_alphafold2_ptm_model_1_seed_000.pdb`

Upload any of the pdb files and the screenshot into this notebook using the files tab to the left. There's a button to upload files. Once uploaded you should press `copy file path` to get the correct paths for your uploads (they usually live under the `/content/` directory).

Display you screenshot and load the pdb using the code provided.

In [ ]:
from IPython.display import Image

def load_pdb_file(file_name):
  pdbp = PDBParser()
  struct = pdbp.get_structure("struct", file_name)
  model = struct[0]

  sequence = []
  coords = []
  for chain in model:
    if chain.id != "A":
      continue
    for residue in chain:
      coords.append(residue['CA'].get_vector().get_array())
      resname = AA_NAME_MAP[residue.resname]
      sequence.append(residue.resname)

  return "".join(sequence), np.array(coords)

def display_image(file_name):
  Image(filename=file_name)

# COMPLETE HERE
# load your structure & display the plot you downloaded
# to get the correct path for your file, select it and press "copy path"
af2_coords = ...

#### 3.B.3 Question

Now fold the same sequence using ESMFold.

You can use this link. Just paste in your sequence and press "fold":  
https://esmatlas.com/resources?action=fold

Once you're done, take a screenshot of the predicted structure and download the PDB file. Upload both files again as you did in the previous problem, display the image here and load the pdb coordinates using the `load_pdb_file` above.

In [ ]:
# COMPLETE HERE
esm2_coords = ...

#### 3.B.4 Question

Now we will compare the two predictions. For this, you will compute the RMSD. Recall the RMSD:

$$ f(x, y) = \frac{1}{N} \sum_{i=1}^N ||x_i - y_i||^2  \\
RMSD(x, y) = \sqrt{f(x, y)} $$


First we will align the two structures and then we will compute the deviation. What is the RMSD between your two predicted structures? Do the models seem to agree? Compare to the ground truth PDB, which model performed best?

In [ ]:
from Bio.SVDSuperimposer import SVDSuperimposer

def compute_rmsd(coords1, coords2):
    """This will align coords2 onto coords1."""
    sup = SVDSuperimposer()
    sup.set(coords1, coords2)
    sup.run()
    rms = sup.get_rms()
    return rms

print(compute_rmsd(af2_coords, esm2_coords))
print(compute_rmsd(af2_coords, true_coords))
print(compute_rmsd(esm2_coords, true_coords))

**ANSWER**:

### 3.C Structure-based sequence sesign (Optional for undegraduates, required for graduate students)

In this problem, we will explore protein sequence design. Here we assume that we have a 3D structure and want to "fit" a sequence of amino acids to it. In practice there are tools you can use to generate protein backbones (ex: RFDiffusion). Here for simplicity, we assume that we have a 3D structure already.

Use ProteinMPNN to redesign the sequence for the PDB file you used in the previous section (ex: 8CH7). All you need to do is run the first few setup cells and in the `Input Options` section, paste in the PDB ID (ex: `pdb: 8CH7`) and the chain to design (ex: `designed_chains: A`). Note that if your PDB has multiple chains (not the case for 8CH7), you may want to use the `fixed chains` options to fix the other chains.

https://colab.research.google.com/github/dauparas/ProteinMPNN/blob/main/colab_notebooks/quickdemo.ipynb

When you are done, you will obtain a new sequence (the one under the text `>T=0.1`).

Past it below and try to fold it using either AF2 or ESMFold.

Compute the RMSD between the new predicted fold and the original ground truth PDB.

#### 3.C.1 Question

How does the newly designed sequence compare to the original one. How does the predicted structures compare to original?



In [ ]:
# COMPLETE HERE
new_sequence = ""
sequence_similarity = ...
rmsd = ...

**ANSWER**: